In [ ]:
#imports
import numpy as np
import matplotlib.pyplot as plt 
from matplotlib.animation import FuncAnimation
from scipy.optimize import curve_fit 
import datetime
from IPython.display import HTML
%matplotlib notebook

In [ ]:
#Exercise 1
#function definitons

def bm_1d(M,pr,dt,dx):
    '''
    

    Parameters
    ----------
    M : TYPE
        DESCRIPTION.
    pr : TYPE
        DESCRIPTION.
    dx : TYPE
        DESCRIPTION.
    dt : TYPE
        DESCRIPTION.

    Returns
    -------
    t : TYPE
        DESCRIPTION.
    x : TYPE
        DESCRIPTION.

    '''
    
    t = np.arange(0,M*dt,dt)
    x = np.zeros(M)
    rand = np.random.choice([-1,1],M,p=[1-pr,pr])
    x = np.cumsum(rand)
    
    return t,x

def bm_mult1(N,M,pr,dt,dx): 
    '''
    

    Parameters
    ----------
    N : TYPE
        DESCRIPTION.
    M : TYPE
        DESCRIPTION.
    pr : TYPE
        DESCRIPTION.
    dx : TYPE
        DESCRIPTION.
    dt : TYPE
        DESCRIPTION.

    Returns
    -------
    t : TYPE
        DESCRIPTION.
    pos : TYPE
        DESCRIPTION.

    '''
    pos = np.zeros([N,M])
    t = np.arange(0,M*dt,dt)
    rand = np.random.uniform(0,1,[N,M])

    for i in range(0,N):
        for j in range(0,M):
            if rand[i][j]<=pr:
                pos[i][j]=pos[i][j-1]+dx
            else:
                pos[i][j]=pos[i][j-1]-dx

    return t, pos

def bm_mult2(N,M,pr,dt,dx):
    '''
    

    Parameters
    ----------
    N : TYPE
        DESCRIPTION.
    M : TYPE
        DESCRIPTION.
    pr : TYPE
        DESCRIPTION.
    dx : TYPE
        DESCRIPTION.
    dt : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    '''
    pos = np.zeros([N,M])
    t = np.arange(0,M*dt,dt)
    rand = np.random.choice([-dx,dx],[N,M],p=[1-pr,pr])
    pos = np.cumsum(rand,axis=0)
    return t, pos.T

bm_mult2(10, 10, 0.5, 1, 1)

def empirical_var(t,x):
    '''
    

    Parameters
    ----------
    t : TYPE
        DESCRIPTION.
    x : TYPE
        DESCRIPTION.

    Returns
    -------
    None.

    '''
    s=np.zeros(len(t))
    x_average = np.zeros(len(t))   
    
    for i in range(0,len(t)):
        res=0
        x_average[i] = np.average(x.T[i])
        res=np.sum((x.T[i]-x_average[i])**2)
        s[i]=(res/(len(x)-1))
    return s

def bm_2d(N,M,pu,pr,dt,dx):
    '''
    

    Parameters
    ----------
    N : TYPE
        DESCRIPTION.
    M : TYPE
        DESCRIPTION.
    pu : TYPE
        DESCRIPTION.
    pr : TYPE
        DESCRIPTION.
    dx : TYPE
        DESCRIPTION.
    dt : TYPE
        DESCRIPTION.

    Returns
    -------
    t : TYPE
        DESCRIPTION.
    pos2D : TYPE
        DESCRIPTION.

    '''
    t = np.arange(0,M*dt,dt);
    pos2D = np.zeros([2,N,M])
    rand_hv = np.random.choice([0,1],[N,M],p=[1-pu,pu]) 
    rand_ud = np.random.choice([-dx,dx],[N,M],p=[1-pr,pr]) 
    rand_lr = np.random.choice([-dx,dx],[N,M],p=[1-pr,pr])
    rand_lr[rand_hv==1]=0
    rand_ud[rand_hv==0]=0
    pos_hv = np.cumsum(rand_ud, axis = 1)
    pos_lr = np.cumsum(rand_lr, axis = 1)
    pos2D[0]=pos_lr
    pos2D[1]=pos_hv

    return t, np.transpose(pos2D,(1,2,0))

def animate_particle(t,pos2D,N,M, pu,pr,lim,colors,dt,dx):
    '''
    

    Parameters
    ----------
    t : TYPE
        DESCRIPTION.
    pos2D : TYPE
        DESCRIPTION.
    dt : TYPE
        DESCRIPTION.
    dx : TYPE
        DESCRIPTION.
    N : TYPE
        DESCRIPTION.
    M : TYPE
        DESCRIPTION.
    pu : TYPE
        DESCRIPTION.
    pr : TYPE
        DESCRIPTION.
    lim : TYPE
        DESCRIPTION.
    colors : TYPE
        DESCRIPTION.

    Returns
    -------
    TYPE
        DESCRIPTION.

    '''
    fig, ax = plt.subplots()
    graph = []
    fig.suptitle(f'Brownian motion of {N} particles with $p_R = {pr}$ and $p_U = {pu}$')
    ax.set_ylim(-lim,lim)
    ax.set_xlim(-lim,lim)
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    for i in range(0,len(pos2D)):
        x=pos2D[i].T[0]
        y=pos2D[i].T[1]
        graph.append(ax.plot(x[0],y[0],marker='o',color=colors[i]))

    def animation_frame(i):
        for k in range(0,len(pos2D)):
            data= (pos2D[k].T[0][i*10],pos2D[k].T[1][i*10])
            graph[k][0].set_data(data)
        
        return graph,
        
    ani = FuncAnimation(fig, func = animation_frame, frames=int(M/10), interval=dt*10)
    plt.show()
    return HTML(ani.to_jshtml())


#---------------------------- Exercise 1h ------------------------------------#

def origo(pos, N, M, x=0,y=0):
    '''

    Parameters
    ----------
    pos : TYPE
        DESCRIPTION.
    N : TYPE
        DESCRIPTION.
    M : TYPE
        DESCRIPTION.

    Returns
    -------
    n : TYPE
        DESCRIPTION.

    '''
    n = np.zeros(M)
    pkt=[x,y]
    for i in range(0,N):    
        temp=pos[i]
        for j in range(1,M):
            if np.all(temp[j]==pkt):
                n[j]=n[j-1]+1
            else:
                n[j]=n[j-1]
    return n



# Oppgave 1a
$$ \frac{\partial}{\partial t}  \varphi  \big(x, t\big) = D  \frac{\partial^2}{\partial x^2} \varphi  \big(x, t\big) $$
hvor D er en konstant. Antar at partikkeltettheten $\varphi$ er en normal sansynlightesfordeling med forventningsverdi  $\mu = 0$, og varians $ \sigma ^{2} = at$. Partikkeltetthet vil da kunne skrives som:  $$ \varphi  \big(x, t\big) = \frac{1}{ \sqrt{2  \pi a t} } exp \big\{- \frac{1}{2}  \frac{ x^{2} }{at} \big\} $$ Partiellderiverer med hennsyn på tid og posisjon gir oss:  $$ \frac{1}{2} = D  \;\;\;\;\;\;\;;\;\;\;\;\;\; \frac{1}{2a} = D \frac{1}{2  a^{2} }  $$ som betyr at $a = 1$

In [ ]:
dt = 1
dx = 1

#---------------------------- Exercise 1b ------------------------------------#


"""
Exercise 1c

"""
pr=[0.45,0.5,0.55]
M = 10000
t,x = bm_1d(M, pr[0],dt,dx);

plt.figure();
plt.plot(t,x, label = f"$p_R = {pr[0]}$");

t,x = bm_1d(M, pr[1],dt,dx);
plt.plot(t,x, label = f"$p_R = {pr[1]}$");

t,x = bm_1d(M, pr[2],dt,dx);
plt.plot(t,x, label = f"$p_R = {pr[2]}$");

plt.title("Brownian motion in 1D")
plt.xlabel("Time [s]")
plt.ylabel("Left / Right")
plt.legend()
    

#--------------------------- Exercise 1d -------------------------------------#

start_time = datetime.datetime.now()

t,x = bm_mult1(1000, 1000, 0.5,dt,dx)

plt.figure();
plt.plot(t,x[0], label = "$Particle 1$");
plt.plot(t,x[1], label = "$Particle 2$");
plt.plot(t,x[2], label = "$Particle 3$");

plt.title("Brownian motion of multiple particles in 1D")
plt.xlabel("Time [s]")
plt.ylabel("Left / Right")
plt.legend()
    
end_time = datetime.datetime.now()

print("Time used with first method (Exercise 1d.)",end_time - start_time)
#-----------------------------------------------------------------------------#



start_time = datetime.datetime.now()

t,x = bm_mult2(1000, 1000, 0.5,dt,dx)

plt.figure();
plt.plot(t,x[0], label = "$Particle 1$");
plt.plot(t,x[1], label = "$Particle 2$");
plt.plot(t,x[2], label = "$Particle 3$");

plt.title("Brownian motion of multiple particles in 1D")
plt.xlabel("Time [s]")
plt.ylabel("Left / Right")
plt.legend()
    
end_time = datetime.datetime.now()
print("Time used with improved method (Exercise 1e.) ",end_time - start_time)

#--------------------------- Exercise 1f -------------------------------------#

def f(x,a,b):
    return a * x + b;

popt, pcov = curve_fit(f, t, x[0])

plt.figure();
plt.plot(t,x[0], label = "$Motion$");
plt.plot(t,f(t,popt[0],popt[1]), label = "Linear fit")    
plt.title("Brownian motion of particle in 1D with linear fit")
plt.xlabel("Time [s]")
plt.ylabel("Left / Right")
plt.legend()


empirical_var(t, x)

#---------------------------- Exercise 1g ------------------------------------#
colors = ['red','green','blue', 'orange']

N=4
M=100

pu=0.5
pr=0.5

t,pos2D = bm_2d(N,M,pu,pr,dt,dx)
plt.figure()
plt.title(f"Brownian motion of {N} particles in 2D")
plt.xlabel("x")
plt.ylabel("y")
for i in range(0,N):
    plt.plot(pos2D[i].T[0],pos2D[i].T[1],color=colors[i])
    plt.plot(pos2D[i][-1][0],pos2D[i][-1][1],marker='o',color=colors[i])

animate_particle(t,pos2D, N, M, pu, pr, 60,colors,dt,dx)



In [ ]:
#!!!!!!!!!!!!!!!!!!!!!!!Put in separate blocks!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!#
dt=1
dx=1
N=4
M=100

pu=0.2
pr=0.8
t,pos2D = bm_2d(N,M,pu,pr,dt,dx)

plt.figure()
plt.title(f"Brownian motion of {N} particles in 2D")
plt.xlabel("x")
plt.ylabel("y")
for i in range(0,N):
    plt.plot(pos2D[i].T[0],pos2D[i].T[1],color=colors[i])
    plt.plot(pos2D[i][-1][0],pos2D[i][-1][1],marker='o',color=colors[i])

animate_particle(t,pos2D, N, M, pu, pr, 60,colors,dt,dx)


# Oppgave 1h
Viktig anntakelse: Isometri!$$ $$
$P(x=0, t=1)$ vil være $0$ uansett dimensjon. Du gjør et tidssteg unna origin. Det minste antallet steg som trengs for å kunne ha $n > 0$ er $2$ steg. $$ $$
$P(x=0, t=2)$ er nå ikke lenger $0$, men sansynligheten vil variere etter dimensjonene og tiltatte bevegelser. 
I $1D$ er det kunn $2$ tiltatte retninger (Høyre / Venstre) så for at $n=1$ må steg $2$ være det omvendte av steg 1: enten høyre - venstre, eller venstre- høyre. Sansynligheten for hvær altærnativ er $0.5  \cdot 0.5$ og med 2 altærnativer er $P = 2 \cdot0.5 \cdot0.5 = 0.5$ $$ $$
I $2D$ er det nå 4 tiltatte retninger (Øst, Vest, Nord, Sør). Hvis første steg var nordover må andre steg være sør som utgjør en fjærdel. Vi har 4 mulige altærnativer hver med $0.25 \cdot0.25$ sansynlighet: $P = 4 \cdot0.25 \cdot0.25 = 0.25$

In [ ]:
#---------------------------- Exercise 1i ------------------------------------#

start_time = datetime.datetime.now()

N=20
M=10000
pu = 0.5
pr = 0.5
plt.figure()
plt.title('Number of times a particle passes point [0,0] ')
t,pos = bm_mult2(N,M,pr,dt,dx)
n = origo(pos.T,N,M)
plt.plot(t,n,'b',label="1D motion")

t,pos2d = bm_2d(N,M,pu,pr,dt,dx)
n2d = origo(pos2d,N,M)
plt.plot(t,n2d,'r',label="2D motion")

plt.xlabel("Time [s]")
plt.ylabel("Number of times passed origo")
plt.legend()

end_time = datetime.datetime.now()
print("Time used (Exercise 1i.)",end_time - start_time)

In [ ]:
#Exercise 2
#Function definitions

def ind(pos,steps,L):
    '''
    Calculates the index of a given position in a numpy mesh grid with
    dimensions [steps,steps] 

    Parameters
    ----------
    pos : ndarray,shape(2,)
        meshgrid position
    steps : int
        discretization of the meshgrid
    L : float
        Limits of the grid.
    Returns
    -------
        int
        index of the point in the meshgrid.

    '''
    
    stepsize = (2*L/steps)
    return int((pos+L)/stepsize)


def create_tumors(xv,yv,t_area,t_centers,n,dx):
    '''
    Creates a meshgrid with zones with different stepsize (tumors)

    Parameters
    ----------
    xv : ndarray
        meshgrid x.
    yv : ndarray
        meshgrid y.
    t_area : float
        area of the tumors.
    t_centers : ndarray, shape(n,2)
        positions of the centers of the tumor.
    n : int
        number of tumors.

    dx : float
        default value of dx outside of tumors


    Returns
    -------
    dx_arr : ndarray, same shape as xv and yv
        meshgrid with dx values.

    '''
    dx_arr=(xv**0+yv**0)*dx/2

    t_coeff = np.random.uniform(0.3,0.45,[n])
    t_radius = np.sqrt(t_area*10/np.pi)
    inside = []
    for i in range(0,n):
        inside.append(np.sqrt((xv-t_centers[i][0])**2+(yv-t_centers[i][1])**2)<=t_radius)
        dx_arr[inside[i]]*=np.sqrt(t_coeff[i])
    return dx_arr


def bm_step(pos,xv,yv,N,pu,pr,dt,dx0,dx_arr,steps,L): 
    '''
    Calculates a step the brownian motion for N particles 

    Parameters
    ----------
    pos : ndarray, shape(N,2)
        initial positions of the particles.
    xv : ndarray, shape(steps,steps)
        meshgrid x.
    yv : ndarray, shape(steps,steps)
        meshgrid y.
    N : int
        number of particles.
    pu : float [0,1]
        probability to go vertical (vs. horisontal).
    pr : float [0,1]
        probablity to go right(up if vertical).
    dt : float
        timestep
    dx0 : float
        stepsize, default outside tumors
    dx_arr : ndarray(steps,steps)
        stepsizes array.
    steps : int
        discretization constant of the meshgrid.
    L : float
        limits of the work area.

    Returns
    -------
    pos : ndarray(N,2)
        Positions array for N number of particles.
    hits : int
        number of times a particle have been inside a tumor .

    '''
    rand_hv = np.random.choice([0,1],[N], p=[1-pu,pu]) 
    rand_ud = np.random.choice([-1,1],[N], p=[1-pr,pr]) 
    rand_lr = np.random.choice([-1,1],[N], p=[1-pr,pr])
    mod_ud=np.ones(N)
    mod_lr=np.ones(N)
    hits = []
    for i in np.arange(0,N):

        if rand_hv[i]==1:
            rand_lr[i]=0
        if rand_hv[i]==0:
            rand_ud[i]=0
        mod_ud[i] = rand_ud[i]*dx_arr[ind(pos[i][0],steps,L)][ind(pos[i][1],steps,L)]
        mod_lr[i] = rand_lr[i]*dx_arr[ind(pos[i][0],steps,L)][ind(pos[i][1],steps,L)]
        if abs(pos[i][0]+mod_lr[i])>=abs(L):
            # pos[i][0]*=-1
            mod_lr*=0
            mod_ud*=0

        if abs(pos[i][1]+mod_ud[i])>=abs(L):
            # pos[i][1]*=-1
            mod_lr*=0
            mod_ud*=0
        if dx_arr[ind(pos[i][0],steps,L)][ind(pos[i][1],steps,L)] != dx0:
            hits.append([pos[i][0],pos[i][1]])
            
    pos_change=np.array([mod_lr,mod_ud])
    pos=pos + pos_change.T

    return pos,hits

def bm_different_dx(M,N,xv,yv,pu,pr,dt,dx,dx0,steps,L):
    '''
    calculates the brownian motion for N particles and M timesteps

    Parameters
    ----------
    M : int
        timesteps.
    N : int
        number of particles.
    xv : ndarray, shape(steps,steps)
        meshgrid x.
    yv : ndarray, shape(steps,steps)
        meshgrid y.
    N : int
        number of particles.
    pu : float [0,1]
        probability to go vertical (vs. horisontal).
    pr : float [0,1]
        probablity to go right(up if vertical).
    dt : float
        timestep
    dx0 : float
        stepsize, default outside tumors
    dx : ndarray(steps,steps)
        stepsizes array.
    steps : int
        discretization constant of the meshgrid.
    L : int
        limits of the work area.

    Returns
    -------
    t : ndarray, shape(M,)
        array with times.
    ndarray, shape(N,M,2)
        path of the particles.
    ndarray, shape(h,2), where h is the number of hits
        ndarray with the positions of where the particles have been in a tumor.

    '''
    
    hits=[]
    t = np.arange(0,M*dt,dt);
    
    pos = np.zeros([M,N,2])
    pos0 = [[-1/2*L,-1/2*L],[-1/2*L,1/2*L],[1/2*L,-1/2*L],[1/2*L,1/2*L]]
    pos[::][0]+=pos0
    
    for i in np.arange(1,M):
        pos[i], h = bm_step(pos[i-1],xv,yv,N,pu,pr,dt,dx0,dx,steps,L)      
        hits.extend(h)
    return t, np.transpose(pos,(1,0,2)),np.array([hits])[0].T

def intensity (pos,xmin,xmax,ymin,ymax,nx,ny):
    '''
    Calculates the intensity of individual particle motion

    Parameters
    ----------
    pos : ndarray
        path of the particle.
    xmin : float
        lower limit of x.
    xmax : float
        upper limit of x.
    ymin : float
        lower limit of y.
    ymax : float
        upper limit of y.
    nx : int
        number of elements in the x direction.
    ny : int
        number of elements in the y direction.

    Returns
    -------
    I : ndarray
        matrix with intensity values of individual particle.
    xedges : ndarray, shape(nx+1,)
        The bin edges along the first dimension.

    yedges : ndarray, shape(ny+1,)
        The bin edges along the second dimension.

    '''
    I, xedges, yedges = np.histogram2d(pos[0],pos[1],bins=(nx,ny),range=([xmin, xmax], [ymin, ymax]))
    return I,xedges, yedges

def intensity_all(pos_arr,xmin,xmax,ymin,ymax,nx,ny,N):
    '''
    Sums over the intensity of N number particles

    Parameters
    ----------
    pos_arr : ndarray
        position vector of pos_arr.
    xmin : float
        lower limit of x.
    xmax : float
        upper limit of x.
    ymin : float
        lower limit of y.
    ymax : float
        upper limit of y.
    nx : int
        number of elements in the x direction.
    ny : int
        number of elements in the y direction.
    N : int
        number of particles.

    Returns
    -------
    I : ndarray, shape(n,m)
        Comulative intensity of all particles motion.
    xedges : ndarray, shape(nx+1,)
        The bin edges along the first dimension.
    yedges : ndarray, shape(ny+1,)
        The bin edges along the second dimension.

    '''
    I, xedges, yedges = intensity(pos_arr[0].T,xmin,xmax,ymin,ymax,nx,ny)
    for i in range (1,N):
        H,xedges,yedges=intensity(pos_arr[i].T,xmin,xmax,ymin,ymax,nx,ny)
        I+=H
    return I, xedges, yedges


def Sobel(n,m,I):
    '''
    Applies a Sobel filter on a Intensity matrix

    Parameters
    ----------
    n : int
        number of elements in x direction.
    m : int
        number of elements in y direction.
    I : ndarray, shape (n,m)
        Intensity.

    Returns
    -------
    S : ndarray, shape(n,m)
        Intensity with applied Sobel filter.
    '''
    g_x = np.matrix("1 0 -1; 2 0 -2; 1 0 -1")
    g_y = np.matrix("1 2 1; 0 0 0; -1 -2 -1")

    X = np.zeros((n-2, m-2))
    Y = np.zeros((n-2, m-2))
    S = np.zeros((n-2, m-2))

    for i in range(2, n-1):
        for j in range(2, m-1):
            A_local = np.array([[I[i-1][j-1], I[i-1][j], I[i-1][j+1]], [I[i][j-1], I[i][j], I[i][j+1]], [I[i+1][j-1], I[i+1][j], I[i+1][j+1]]])
           
            X[i-1][j-1] = np.sum(np.multiply(g_x, A_local))
            Y[i-1][j-1] = np.sum(np.multiply(g_y, A_local))
            S[i-1][j-1] = np.sqrt((X[i-1][j-1])**2 + (Y[i-1][j-1])**2)
    
    #normalization of X, Y and S
    X = X / np.max(X)
    Y = Y / np.max(Y)
    S = S / np.max(S)
    
    return S


# Oppgave 2a
$$D = \frac{   \Delta x^{2} }{ 2\Delta t}$$
der $\Delta x = 0.004 mm $, $\Delta t = 0.01 s $ i et friskt vev $$ $$ Om vi introduserer $i$ tumorer med koeffisienter $ \kappa_{i} $ vil de redusere $\Delta x$ og dermed Disfusjonskonstanten på følgende måte: 
$$ D = \frac{ \big(\Delta x- \sqrt{ \prod_{i}   \kappa _{i}  }\big) ^{2} }{ 2\Delta t} $$

In [ ]:
dx0 = 0.4 #[microm]
dt = 0.01  #[s]

n = 25     #number of tumors
L = 20     #[microm]  #focus_le
steps = 1000
stepsize = 2*L/steps

N=4
M=10000

pu=0.5
pr=0.5

x =np.linspace(-L,L,steps)
y =np.linspace(-L,L,steps)
xv,yv = np.meshgrid(x,y,indexing='xy')
t_area = dx0**2*np.pi #area of tumor
t_centers = np.random.uniform(-L+np.sqrt(t_area/np.pi), L-np.sqrt(t_area/np.pi),(n,2))


dx_arr = create_tumors(xv, yv, t_area, t_centers, n, dx0)
t,pos,hits= bm_different_dx(M,N,xv,yv,pu,pr,dt,dx_arr,dx0,steps,L)

I,xedges,yedges=intensity_all(pos,-L,L,-L,L,100,100,N)

I=I/(M*N)

sobel = Sobel(100,100,I)

    
#PLOTS
plt.figure()
plt.contourf(xv,yv,dx_arr, cmap='binary')
if hits.size>0:
    plt.plot(hits[1],hits[0], marker='o',color='r',linestyle= 'None', markersize = 0.1)
plt.plot([],[],marker='o',color='r',linestyle= 'None',label=f'Tumor hits $n={hits[0].size}$')
plt.title("Tumor positions", fontsize=24 )
plt.xlabel('x $[\mu m]$')
plt.ylabel('y $[\mu m]$')
plt.legend()

extent = [xedges[0], xedges[-1], yedges[-1], yedges[0]]

fig = plt.figure()
fig.suptitle("Intensity",fontsize=24)
fig.tight_layout()

ax = fig.add_subplot(111)    # The big subplot
ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)

ax1.set_title("Unfiltered")
ax1.imshow(I, extent=extent,cmap='gray')
ax1.invert_yaxis()

ax2.set_title("Sobel filter")
ax2.imshow(sobel, extent=extent,cmap='gray')
ax2.invert_yaxis()


# Turn off axis lines and ticks of the big subplot
ax.spines['top'].set_color('none')
ax.spines['bottom'].set_color('none')
ax.spines['left'].set_color('none')
ax.spines['right'].set_color('none')
ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)

ax.set_xlabel('x $[\mu m]$')
ax.set_ylabel('y $[\mu m]$')
#-------------------------------------------------------------------------
